In [ ]:
import keras_nlp
import transformers
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
 

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
tf.config.list_physical_devices('GPU')

# Data preparation

In [ ]:
train_df = pd.read_csv("../dataset/train.csv")
test_df = pd.read_csv("../dataset/test.csv")
titles = pd.read_csv("../dataset/titles.csv")

In [ ]:
titles = titles.loc[titles['code'].str.len() == 3]

In [ ]:
train_df['context_text'] = train_df.apply(lambda r: titles[(titles['code'] == r['context'])]['title'].iloc[0], axis=1)

In [ ]:
train_df['text'] = train_df['anchor']+";"+train_df['target']+";"+train_df['context_text']

In [ ]:
train_df

In [ ]:
train_df.score.value_counts()

In [ ]:
max_length= 16

# Data generator

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
    """ Generates batches of data.
    Args:
        phrases: Array of anchor, target and contex input phrases.
        scores: array of similarity scores.
        batch_size: Integer batch size.
        shuffle: boolean, whether to shuffle the data.
        include_targets: boolean, whether to include the labels.
    Returns:
        Tuples '([input_ids, attention_mask, token_type_ids], labels)'
        or just '[input_ids, attention_mask, token_type_ids]'
        if 'include_target=False'"""
    
    def __init__(self, phrases, scores, batch_size=10, shuffle=True, include_targets=True):
        self.phrases = phrases
        self.scores = scores
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.include_targets = include_targets
        
        self.tokenizer = transformers.BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)
        self.indexes = np.arange(len(self.phrases))
        self.on_epoch_end()
    
    def __len__(self):
        # Denotes the number of batches per epoch.
        return len(self.phrases) // self.batch_size
    
    def __getitem__(self, idx):
        # Retrieves the batch of index.
        indexes = self.indexes[idx * self.batch_size : (idx+1) * self.batch_size]
        phrases = self.phrases[indexes]
        
        # With BERT tokenizer's batch_encode_plus batch of both the phrases and context are
        # encoded together and separated by [SEP] token.
        encoded = self.tokenizer.batch_encode_plus(
            phrases.tolist(),
            add_special_tokens=True,
            max_length=max_length,
            return_attention_mask=True,
            return_token_type_ids=True,
            padding='max_length',
            return_tensors="tf",
            truncation=True
            )
        
        # convert batch of encoded features to numpy array.
        input_ids = np.array(encoded["input_ids"], dtype="int32")
        attention_masks = np.array(encoded["attention_mask"], dtype="int32")
        token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")
        
        # set to true if data generator is used for training/validation.
        if self.include_targets:
            labels = np.array(self.scores[indexes], dtype="int32")
            return [input_ids, attention_masks, token_type_ids], labels
        else:
            return [input_ids, attention_masks, token_type_ids]
        
    def on_epoch_end(self):
        # Shuffle indexes after each epoch if shuffle is set to True.
        if self.shuffle:
            np.random.RandomState(42).shuffle(self.indexes)
            

# Build the model

In [ ]:

# Encoded token ids from BERT tokenizer.
input_ids = tf.keras.layers.Input(
    shape=(max_length,),dtype=tf.int32, name="input_ids"
)

# Attention masks indicates to the model which tokens should be attended to.
attention_masks = tf.keras.layers.Input(
    shape=(max_length,), dtype=tf.int32, name="attention_masks"
)

# Token type ids are binary masks identifying different sequences in the model
token_type_ids = tf.keras.layers.Input(
    shape=(max_length,), dtype=tf.int32, name="token_type_ids"
)

# Loading pretrained BERT model.
bert_model = transformers.TFBertModel.from_pretrained("bert-base-uncased")
# Freeze the BERT model to reuse the pretrained features without modifying them
bert_model.trainable = False

bert_output = bert_model.bert(
input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids)
sequence_output = bert_output.last_hidden_state
pooled_output = bert_output.pooler_output

# Add trainable layers on top of frozen layers to adapt the pretrained features on the data.
bi_lstm = layers.Bidirectional(layers.LSTM(1, return_sequences=True))(sequence_output)
avg_pool = layers.GlobalAveragePooling1D()(sequence_output)
max_pool = layers.GlobalMaxPooling1D()(sequence_output)
add = layers.add([avg_pool, max_pool])
dropout = layers.Dropout(0.3)(add)

# try to treat the scores as labels
output = layers.Dense(1, activation="sigmoid")(dropout)
model = tf.keras.models.Model(
    inputs = [input_ids, attention_masks, token_type_ids], outputs=output
)

model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss="mse",
    metrics=["mse"],
)

model.summary()   

    

In [ ]:
df_randomized = train_df.sample(frac=1, random_state=1)
training_test_index = round(len(df_randomized)*0.8)
train = df_randomized[:training_test_index].reset_index(drop=True)
valid = df_randomized[training_test_index:].reset_index(drop=True)

batch_size = 32
train_data = DataGenerator(
    train[["target","text"]].values.astype("str"),
    train["score"].values,
    batch_size=batch_size,
    shuffle=True,
)
valid_data = DataGenerator(
    valid[["target","text"]].values.astype("str"),
        train["score"].values,
    batch_size=batch_size,
    shuffle=True, 
)

In [ ]:
transformers.logging.set_verbosity_error()

In [ ]:
history=model.fit(train_data,validation_data=valid_data, epochs=10, use_multiprocessing=True, workers=-1)